In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor

training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308491  [    0/60000]
loss: 2.297221  [ 6400/60000]
loss: 2.283693  [12800/60000]
loss: 2.279331  [19200/60000]
loss: 2.265552  [25600/60000]
loss: 2.259570  [32000/60000]
loss: 2.249852  [38400/60000]
loss: 2.231648  [44800/60000]
loss: 2.238460  [51200/60000]
loss: 2.227322  [57600/60000]
Test Error: 
 Accuracy: 44.4%, Avg loss: 0.034837 

Epoch 2
-------------------------------
loss: 2.228489  [    0/60000]
loss: 2.219347  [ 6400/60000]
loss: 2.179409  [12800/60000]
loss: 2.186582  [19200/60000]
loss: 2.158824  [25600/60000]
loss: 2.152693  [32000/60000]
loss: 2.136099  [38400/60000]
loss: 2.101917  [44800/60000]
loss: 2.132697  [51200/60000]
loss: 2.106189  [57600/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 0.032846 

Epoch 3
-------------------------------
loss: 2.115260  [    0/60000]
loss: 2.097089  [ 6400/60000]
loss: 2.016748  [12800/60000]
loss: 2.035261  [19200/60000]
loss: 1.991295  [25600/60000]
loss: 1.987618  [32000/600